1*. Измените функцию calc_logloss так, чтобы нули по возможности не попадали в np.log

In [116]:
import numpy as np
import matplotlib.pyplot as plt
from itertools import product

In [288]:
def calc_logloss(y, y_pred):
    err = - np.mean(y * np.log(y_pred+1e-5) + (1.0 - y) * np.log(1+1e-5- y_pred))
    err = np.sum(err)
    return err

Подберите аргументы функции eval_model для логистической регрессии таким образом, чтобы log loss был минимальным

In [118]:
X = np.array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
              [1, 1, 2, 5, 3, 0, 5, 10, 1, 2],
              [500, 700, 750, 600, 1450, 800, 1500, 2000, 450, 1000],
              [1, 1, 2, 1, 2,  1, 3, 3, 1, 2]], dtype = np.float64)
y = np.array([0, 0, 1, 0, 1, 0, 1, 0, 1, 1], dtype = np.float64)

In [190]:
def calc_std_feat(x):
    res = x.copy()
    for i in range(1,x.shape[0]):
        res[i,:]= (res[i,:] - res[i,:].mean()) / res[i,:].std()
    return res

In [462]:
X_st = calc_std_feat(X)
X_st

array([[ 1.        ,  1.        ,  1.        ,  1.        ,  1.        ,
         1.        ,  1.        ,  1.        ,  1.        ,  1.        ],
       [-0.70710678, -0.70710678, -0.35355339,  0.70710678,  0.        ,
        -1.06066017,  0.70710678,  2.47487373, -0.70710678, -0.35355339],
       [-0.97958969, -0.56713087, -0.46401617, -0.77336028,  0.97958969,
        -0.36090146,  1.08270439,  2.11385144, -1.08270439,  0.05155735],
       [-0.89625816, -0.89625816,  0.38411064, -0.89625816,  0.38411064,
        -0.89625816,  1.66447944,  1.66447944, -0.89625816,  0.38411064]])

In [121]:
def sigmoid(z):
    res = 1 / (1 + np.exp(-z))
    return res

In [444]:
def eval_model(X, y, iterations, alpha=1e-4):
    np.random.seed(42)
    W = np.random.randn(X.shape[0])
    n = X.shape[1]
    for i in range(1, iterations+1):
        z = np.dot(W, X)
        y_pred = sigmoid(z)
        err = calc_logloss(y, y_pred)          
        W -= alpha * (1/n * np.dot((y_pred - y), X.T))
#         if i % (iterations / 10) == 0:
#             print(i, W, err)
    return W, err

In [445]:
iterations = np.arange(1000, 10000, 1000)
alpha =  np.linspace(1e-4, 1, 100)


In [450]:
C = {'alpha':0,'iterations':0, 'err_min': np.inf}
for case in product(alpha, iterations):
    err = eval_model(X_st, y, iterations=case[1], alpha=case[0])[1]
    if C['err_min'] - err>=0.1:
        C['err_min']=err
        C['alpha']=case[0]
        C['iterations']=case[1]
        
C          

{'alpha': 0.3031, 'iterations': 9000, 'err_min': 0.12023881802087388}

Создайте функцию calc_pred_proba, возвращающую предсказанную вероятность класса 1 (на вход подаются W, который уже посчитан функцией eval_model и X, на выходе - массив y_pred_proba).

In [300]:
def calc_pred_proba(W, X):
    return sigmoid(np.dot(W,X))
    
    

In [451]:
W=eval_model(X_st, y, iterations=9000, alpha=0.3)[0]
W

array([ 2.93551763, -8.85809981, -8.51431379, 20.13183143])

In [452]:
calc_pred_proba(W, X_st)

array([3.76684670e-01, 1.77153801e-02, 9.99999980e-01, 3.78597849e-07,
       9.11156176e-01, 6.66367284e-02, 9.99999999e-01, 2.99989186e-02,
       5.92499249e-01, 9.99998425e-01])

Создайте функцию calc_pred, возвращающую предсказанный класс (на вход подаются W, который уже посчитан функцией eval_model и X, на выходе - массив y_pred).

In [453]:
def calc_pred(W,X):
    m = X.shape[1]   
    y_predicted = np.zeros((1, m))
    W = W.reshape(X.shape[0], 1)
    
    A = sigmoid(np.dot(W.T, X))
     
    for i in range(A.shape[1]):
        if (A[:,i] >= 0.5): 
            y_predicted[:, i] = 1
        elif (A[:,i] < 0.5):
            y_predicted[:, i] = 0

    return y_predicted
    

In [454]:
calc_pred(W,X_st)

array([[0., 0., 1., 0., 1., 0., 1., 0., 1., 1.]])

In [455]:
y

array([0., 0., 1., 0., 1., 0., 1., 0., 1., 1.])

Посчитайте Accuracy, матрицу ошибок, точность и полноту, а также F1 score

In [456]:
accur = np.count_nonzero(calc_pred(W,X_st)==y)/len(y)
accur

1.0

In [457]:
matrix_err = np.zeros((2, 2))
matrix_err[0][0]=np.count_nonzero(calc_pred(W,X_st)*y==1)/np.count_nonzero(y==1)
matrix_err[0][1]=1-matrix_err[0][0]
matrix_err[1][1]=1-matrix_err[0][1]
matrix_err[1][0]=1-matrix_err[1][1]
matrix_err

array([[1., 0.],
       [0., 1.]])

In [458]:
prec = matrix_err[0][0]
prec

1.0

In [459]:
rec = matrix_err[0][0]
rec

1.0

In [460]:
f1_score = 2*prec*rec/(prec+rec)
f1_score

1.0

Могла ли модель переобучиться? Почему?

In [461]:
# В случае линейной модели переобучение возможно вследствии мультиколленеарности, признаком чего 
# являются большие значения весов, что в данном случае не наблюдается. Проверить переобучение можно на тестовой выборке,
#  но размер исходной выборки очень мал.